# Import

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
from sklearn.metrics import confusion_matrix, classification_report

pd.set_option('expand_frame_repr', False)
pd.set_option('display.max_colwidth', 0)

%matplotlib inline
print("Complete")

Complete


# Input

In [39]:
filePathDataEval = r'C:\Users\nguyen.tt1\Downloads\eval.csv'
filePathDataTrain = r'C:\Users\nguyen.tt1\Documents\Nanibot_ZaloAIChallenge2019_VietnameseWikiQA\Dataset\train.json'
nTopQAPair = 10

In [40]:
evalData = pd.read_csv(filePathDataEval, encoding="utf8")
trainData = pd.read_json(filePathDataTrain, encoding="utf8")

In [41]:
evalData.insert(
    len(evalData.columns),
    'IsCorrect',
    ~(evalData['prediction'] ^ evalData['label'])
)

# Utilities

In [42]:
def plotConfusionMatrix(cm, normalize=False):
    cmap = plt.get_cmap('Greens')

    plt.imshow(cm,cmap=cmap)
    plt.colorbar()
    plt.xticks([0,1],['True','False'])
    plt.yticks([0,1],['True','False'])
    
    if normalize:
        cm = (cm.astype('float') / cm.sum(axis=1)[:, np.newaxis])
    
    for i, j in product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}%\n".format(cm[i, j]*100),
                    ha="center",
                    va="center",
                    size='large',
                    color="black",
                    weight="bold"
            )
        else:
            plt.text(j, i, "{:,}\n".format(cm[i, j]),
                    ha="center",
                    va="center",
                    size='large',
                    color="black",
                    weight="bold"
            )

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
    if normalize:
        plt.title("Confusion matrix - Normalized\n\n")
    else:
        plt.title("Confusion matrix\n\n")
    plt.show()

In [43]:
def getTopQAPair(nQAPair, useCorrectPrediction=True,sortAsc=True):
    if (useCorrectPrediction):
        topQA = evalData[
            evalData['IsCorrect']
        ].sort_values(by=['probabilities'], ascending=sortAsc).iloc[:nQAPair][['guid','prediction','probabilities']]
    else:
        topQA = evalData[
            ~evalData['IsCorrect']
        ].sort_values(by=['probabilities'], ascending=sortAsc).iloc[:nQAPair][['guid','prediction','probabilities']]

    return pd.merge(
        right=trainData,
        left=topQA,
        right_on='id',
        left_on='guid'
    )[['id','question','title','text','label','prediction','probabilities']]

# Confusion Matrix

In [ ]:
plotConfusionMatrix(
    confusion_matrix(
            evalData['label'],
            evalData['prediction']
    )
)
plotConfusionMatrix(
    confusion_matrix(
            evalData['label'],
            evalData['prediction']
    ),
    normalize=True
)
print("Classification report\n")
print(classification_report(
    evalData['label'],
    evalData['prediction']
))

# Top x QA pairs with the most confidence & correct

In [ ]:
getTopQAPair(nTopQAPair, useCorrectPrediction=True, sortAsc=False)

# Top x QA pairs with the least confidence but correct

In [ ]:
getTopQAPair(nTopQAPair, useCorrectPrediction=True, sortAsc=True)

# Top x QA pairs with the most confidence but incorrect

In [ ]:
getTopQAPair(nTopQAPair, useCorrectPrediction=False, sortAsc=False)

# Top x QA pairs with the least confidence & incorrect

In [ ]:
getTopQAPair(nTopQAPair, useCorrectPrediction=False, sortAsc=True)